In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
#waseda_username="#######"
#keypass="#######"
search_keyword="千の証言： OR 戦後７０年： OR 戦後７０年これまで・これから： OR 戦後７０年・戦後７０年の夏： OR 特集ワイド： OR 会いたい・戦後７０年の夏に"
dffile=[]
#f=open("MainichiChecker.txt","w",encoding="utf-8")
#早稲田検索ページを開ける
browser = webdriver.Chrome()
#browser.get("http://waseda.summon.serialssolutions.com/jp/search?q=%E6%AF%8E%E6%97%A5%E6%96%B0%E8%81%9E&l=jp#!/search?ho=t&l=jp&q=%E6%AF%8E%E6%97%A5%E6%96%B0%E8%81%9E")
#time.sleep(3)
#browser.find_element_by_xpath('//*[@ng-bind="::item.bet.title"]').click()
#print("1.検索完了")
#新しいページでログインする
#allHandles = browser.window_handles
#for handle in allHandles:
#    if browser.title.find("EZproxy") == -1:
#        browser.switch_to_window(handle)

#学外アクセス
#elem_user = browser.find_element_by_name("user")
#elem_user.send_keys(waseda_username)
#elem_pwd = browser.find_element_by_name("pass")
#elem_pwd.send_keys(keypass)
#browser.find_element_by_xpath('//*[@type="submit"]').click()
#time.sleep(2)
#browser.find_element_by_xpath('//div[@class="item clr"]//a[@target="WMSK"]').click()

browser.get("https://dbs.g-search.or.jp/WMAI/WMAI_ipcu_login.html")

browser.get("https://dbs.g-search.or.jp/aps/WMSK/main.jsp?uji.verb=GSHWA0300&serviceid=WMSK")
time.sleep(3)
print("2.ログイン完了")

#キーワード入力
elem_inkw = browser.find_element_by_name("paraTi")
elem_inkw.send_keys(search_keyword)

#期間限定
Select(browser.find_element_by_name("paraYearFrom")).select_by_value("2015") 
Select(browser.find_element_by_name("paraMonthFrom")).select_by_value("1") 
Select(browser.find_element_by_name("paraDayFrom")).select_by_value("1")
Select(browser.find_element_by_name("paraFromTo")).select_by_value("1")
Select(browser.find_element_by_name("paraYearTo")).select_by_value("2015") 
Select(browser.find_element_by_name("paraMonthTo")).select_by_value("12") 
Select(browser.find_element_by_name("paraDayTo")).select_by_value("31")

#見出しだけ検索
browser.find_element_by_xpath('//*[@value="34"]').click()

#東京朝刊＆東京夕刊限定
browser.find_element_by_xpath('//*[@value="東京朝刊"]').click()
browser.find_element_by_xpath('//*[@value="東京夕刊"]').click()

time.sleep(1)
browser.find_element_by_name("btn1Top").click()


#検索結果一覧表示
browser.find_element_by_xpath('//*[@name="paraSort" and @value="4"]').click()
Select(browser.find_element_by_name("paraHyoujiKensuu")).select_by_value("200")
w_news=browser.find_element_by_xpath('//*[@class="num"]').text
browser.find_element_by_xpath('//*[@value="一覧表示"]').click()

#一括表示&テキストスクレイピングのプログラム
def newsdownload(browser,nl):
    #検索結果一括表示
    browser.find_element_by_xpath('//*[@alt="全て選択"]').click()
    browser.find_element_by_xpath('//*[@alt="一括表示"]').click()
    
    #コンテンツ収集
    ttllist=browser.find_elements_by_class_name("title")
    arclist=browser.find_elements_by_class_name("article")
    
    for content,article in zip(ttllist,arclist):
        content=content.text.split("\n")
        ttl=content[0]
        ctt=content[1]
        ctt=ctt.split("　")
        date,area,size=ctt[0],ctt[1],ctt[-1]
        if "頁" in ctt[2]:
            page=ctt[2]
        else:
            page=""
        #if "頁" or "地方版" in ctt[-2]:
        #    men=""
        #else:
        men=ctt[-2]
        nl.append([date.replace(".","/"),ttl,area,men,page,size,article.text])
        #print(ttl+"\n")
        #print(date+"\n")
        #print(article.text)
        #f.write("<H3>"+ttl+"</H3>"+"\n")
        #f.write("<H4>"+date+"</H4>"+"\n")
        #f.write("\n"+article.text+"\n\n\n")
        
    #一覧に戻る
    browser.find_element_by_xpath('//*[@onclick="execute(\'sys_form\',\'Back\',\'C\')"]').click()

#ページの切り替え
w_start=1
w_page=int(w_news)//200+1
while w_start<=w_page:
    newsdownload(browser,dffile)
    nextpage="//*[@onclick=\"javascript:execute(\'sys_form\', \'Page\', \'"+str(w_start+1)+"\');return false;\"]"
    try:
        browser.find_element_by_xpath(nextpage).click()
    except:
        pass
    w_start+=1

browser.quit()
#f.close()

#データフレーム構築
df = pd.DataFrame(dffile,columns=["時間","タイトル","朝夕刊","面","ページ","文字数","記事内容"])
#df.to_csv("C_MainichiCheck.csv",encoding="utf-8",sep="\t", index=False)
df.to_excel('Mainichi.xlsx', sheet_name='sheet1')
df

2.ログイン完了


,時間,タイトル,朝夕刊,面,ページ,文字数,記事内容
0,2015/01/05,特集ワイド：この国はどこへ行こうとしているのか 巨大与党の下で 宗教学者・山折哲雄さん,東京夕刊,総合面,2頁,"(全2,822字)",◇「ひとりの哲学」で歴史を見よ－－山折哲雄さん（８３）\nしっとりした冬の雨が京都をぬらして...
1,2015/01/06,特集ワイド：この国はどこへ行こうとしているのか 巨大与党の下で 憲法学者・木村草太さん,東京夕刊,総合面,2頁,"(全2,805字)",◇国民全体が挑戦受けている－－木村草太さん（３４）\n東京都八王子市の首都大学東京の研究室の...
2,2015/01/07,特集ワイド：この国はどこへ行こうとしているのか 巨大与党の下で 経済学者・伊東光晴さん,東京夕刊,総合面,2頁,"(全2,786字)",◇「戦後の自由」を諦めない－－伊東光晴さん（８７）\n「心拍数が４０程度で、成人男性の正常値...
3,2015/01/08,特集ワイド：この国はどこへ行こうとしているのか 巨大与党の下で 作家・中島京子さん,東京夕刊,総合面,2頁,"(全2,781字)",◇４年で怖いほど変わる－－中島京子さん（５０）\n次の衆院選まで長くて４年。そんな話になった...
4,2015/01/09,特集ワイド：この国はどこへ行こうとしているのか 巨大与党の下で 映画監督・山田洋次さん,東京夕刊,総合面,2頁,"(全3,232字)",◇差別の先に戦争が来る－－山田洋次さん（８３）\n東京都中央区の松竹本社に映画「男はつらいよ...
5,2015/01/13,特集ワイド：社内失業しないためにも…今年こそ！独学,東京夕刊,総合面,2頁,"(全2,546字)",今年こそ何か学びたい！ そんな思いで新年を迎えた人も多いのではないか。今、「独学」が注目され...
6,2015/01/13,「論争」の戦後７０年：第９回 英語教育 児童に強制、共に批判,東京朝刊,解説面,11頁,"(全3,119字)",◇実用か教養か、平行線両氏\n１９７４年４月、元外交官で当時自民党参院議員だった平泉渉氏（８...
7,2015/01/13,千の証言：スパイ養成、陸軍中野学校を直前に卒業 「秘密戦士」むなしい敗戦 使わぬ兵器背負い帰郷,東京朝刊,社会面,27頁,"(全1,778字)",終戦の前日、名古屋市の軍司令部から武器を持てるだけ持ち出した。翌日、玉音放送を聞いたが信用し...
8,2015/01/14,特集ワイド：出版不況や「コンビニ化」のなか――がんばれ！僕らの「本屋」さん,東京夕刊,総合面,2頁,"(全2,599字)",書店でじっくり本を選ぶのが至上の喜び――。そんな本好きの人たちから「最近、本屋がつまらない」...
9,2015/01/15,特集ワイド：民主党代表選、影薄く,東京夕刊,総合面,2頁,"(全2,671字)",文字通りの崖っ縁である。昨年末の衆院選で、トップが落選の憂き目にあった民主党だ。今は新たなト...
